Tests for mod_mod algo

In [1]:
import torch
from torch import distributions as dist

import itertools
import pickle
import importlib

import itertools
import random
import math
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('white')
sns.set_context('paper')
# Plot adjustments:
plt.rcParams.update({'ytick.labelsize': 15})
plt.rcParams.update({'xtick.labelsize': 15})
plt.rcParams.update({'axes.labelsize': 35})
plt.rcParams.update({'legend.fontsize': 30})
plt.rcParams.update({'axes.titlesize': 16})

from gptorch import kernels, models
import helpers, opt, objectives

In [2]:
with open('../inputs/gb1.pkl', 'rb') as f:
    t = pickle.load(f)
X = t[0]
A = t[1]
y = t[2]
wt = t[3]

In [10]:
np.random.seed(120120)
_ = torch.manual_seed(43298)

seq_to_x = {}
for i, x in enumerate(X):
    seq = helpers.decode_X(x)
    seq_to_x[seq] = i
wt_inds = [seq_to_x[wt]]

n = 100
train_inds = wt_inds + list(np.random.choice(len(X), n, replace=True))

y_train = y[train_inds]
y_true = y
A_train = A[train_inds]
A_test = A

inds = sorted(set(itertools.chain(train_inds)))


dic, _ = helpers.get_predictions(A[inds], y[inds], A_test,
                                         one_hots=X, its=3000, lr=1e-3)

Iteration 3000 of 3000	NLML: 49.9795	sn: 0.495660	

In [11]:
aas = 'ARNDCQEGHILKMFPSTWYV'
ground = [(aa, i) for aa in aas for i in range(4)]

In [17]:
seen_seqs = [helpers.decode_X(X[i]) for i in inds]
for s in seen_seqs:
    dic[s] = 0.0

In [18]:
seed = helpers.get_seed(dic)
seed

[('W', 0), ('N', 1), ('Q', 2), ('W', 3)]

In [78]:
%%time

opt = importlib.reload(opt)
objectives = importlib.reload(objectives)
helpers = importlib.reload(helpers)

np.random.seed(10645)

X_list, obj_list = opt.mod_mod(ground, seed, objectives.obj_LHS, objectives.obj_RHS, 
                               fn_args=(4, dic), g_args=(4, dic, 100), verbose=True)

Iteration 0	 obj = -0.000105	
Iteration 1	 obj = -0.003231
Iteration 2	 obj = -0.007021
Iteration 3	 obj = -0.008408
Iteration 4	 obj = -0.009028
Iteration 5	 obj = -0.009695
Iteration 6	 obj = -0.009872
Iteration 7	 obj = -0.010028
Iteration 8	 obj = -0.010114
Iteration 9	 obj = -0.010169
Iteration 10	 obj = -0.010197
Iteration 11	 obj = -0.010223
Iteration 12	 obj = -0.010272
Iteration 13	 obj = -0.010291
Iteration 14	 obj = -0.010308
Iteration 15	 obj = -0.010322
Iteration 16	 obj = -0.010334
Iteration 17	 obj = -0.010339
Iteration 18	 obj = -0.010344
Iteration 19	 obj = -0.010347
Iteration 20	 obj = -0.010348
Iteration 21	 obj = -0.010349
Iteration 22	 obj = -0.010350
Iteration 23	 obj = -0.010350
Iteration 24	 obj = -0.010326
Iteration 25	 obj = -0.010357
Iteration 26	 obj = -0.010361
Iteration 27	 obj = -0.010365
Iteration 28	 obj = -0.010368
Iteration 29	 obj = -0.010370
Iteration 30	 obj = -0.010359
Iteration 31	 obj = -0.010377
Iteration 32	 obj = -0.010386
Iteration 33	 obj =

In [104]:
objectives = importlib.reload(objectives)
opt = importlib.reload(opt)
helpers = importlib.reload(helpers)


opt.greedy(ground, X_list[23], objectives.objective, 1, obj_args=(4, dic, 100))

([('W', 0),
  ('C', 0),
  ('I', 0),
  ('P', 0),
  ('R', 0),
  ('L', 0),
  ('Q', 0),
  ('G', 0),
  ('Y', 0),
  ('E', 0),
  ('K', 0),
  ('F', 0),
  ('N', 0),
  ('V', 0),
  ('N', 1),
  ('R', 1),
  ('G', 1),
  ('F', 1),
  ('S', 1),
  ('K', 1),
  ('D', 1),
  ('C', 1),
  ('L', 1),
  ('Q', 2),
  ('G', 2),
  ('V', 2),
  ('W', 3),
  ('R', 3),
  ('E', 3),
  ('K', 3),
  ('C', 3),
  ('H', 3),
  ('I', 3),
  ('Y', 3),
  ('L', 3),
  ('F', 3),
  ('S', 3),
  ('G', 3),
  ('A', 3),
  ('P', 3),
  ('M', 3)],
 tensor(1.00000e-02 *
        -1.0350))

In [87]:
objectives = importlib.reload(objectives)
opt = importlib.reload(opt)
helpers = importlib.reload(helpers)


%time lib, obj = opt.greedy(ground, seed, ind_obj.objective, 2)
obj

CPU times: user 2min 21s, sys: 1.29 s, total: 2min 22s
Wall time: 2min 24s


tensor(1.00000e-02 *
       -1.0393)

In [23]:
# Check that cant add or take away anything to increase objective

objects = [] # store obj_LHS, obj_RHS, difference btw the two
lib = X_list[-1]
for i in ground:
    if i in lib:
        X_noi = [x for x in lib if x != i]    
        obj_left = objectives.obj_LHS(X_noi, 4, dic)
        obj_right = objectives.obj_RHS(X_noi, 4, dic, 100)
        obj = obj_left - obj_right
        
        objects.append((obj_left, obj_right, obj))
    else:        
        obj_left = objectives.obj_LHS(lib + [i], 4, dic)
        obj_right = objectives.obj_RHS(lib + [i], 4, dic, 100)
        obj = obj_left - obj_right
        objects.append((obj_left, obj_right, obj))

In [24]:
for obj in objects:
    if obj[2] < objectives.objective(lib, 4, dic, 100):
        print(obj)

In [25]:
helpers.get_N(lib, 4)

5600

In [26]:
with open('../outputs/20180828_modmod_not_mono.pkl', 'wb') as f:
    pickle.dump((X_list, obj_list), f)

In [88]:
with open('../outputs/20180828_modmod_not_mono.pkl', 'rb') as f:
    X_list, obj_list = pickle.load(f)

In [116]:
for a in X_list[23]:
    if a not in X_list[24]:
        print(a)
        
for a in X_list[24]:
    if a not in X_list[23]:
        print(a)

('F', 2)


In [113]:
len(X_list[24])

42

In [106]:
inds = [i for i in range(len(objs) - 1) if objs[i] < objs[i + 1]]
inds

[23, 29]

In [109]:
objects = [] # store obj_LHS, obj_RHS, difference btw the two
lib = X_list[29]
n = 100
L = 4
for i in ground:
    if i in lib:
        A = [x for x in lib if x != i]    
    else:        
        A = lib + [i]
    objects.append(objectives.objective(A, L, dic, n))
    
for obj in objects:
    if obj < objectives.objective(lib, 4, dic, 100):
        print(obj)

In [118]:
objectives.objective(X_list[23], L, dic, n)

tensor(1.00000e-02 *
       -1.0326)

In [121]:
%%time

opt = importlib.reload(opt)
objectives = importlib.reload(objectives)
helpers = importlib.reload(helpers)

np.random.seed(10645)

X_list, obj_list, perm_list = opt.mod_mod(ground, seed, objectives.obj_LHS, objectives.obj_RHS, 
                               fn_args=(4, dic), g_args=(4, dic, 100), verbose=True)

Iteration 0	 obj = -0.000105	
Iteration 1	 obj = -0.003231
Iteration 2	 obj = -0.007021
Iteration 3	 obj = -0.008408
Iteration 4	 obj = -0.009028
Iteration 5	 obj = -0.009695
Iteration 6	 obj = -0.009872
Iteration 7	 obj = -0.010028
Iteration 8	 obj = -0.010114
Iteration 9	 obj = -0.010169
Iteration 10	 obj = -0.010197
Iteration 11	 obj = -0.010223
Iteration 12	 obj = -0.010272
Iteration 13	 obj = -0.010291
Iteration 14	 obj = -0.010308
Iteration 15	 obj = -0.010322
Iteration 16	 obj = -0.010334
Iteration 17	 obj = -0.010339
Iteration 18	 obj = -0.010344
Iteration 19	 obj = -0.010347
Iteration 20	 obj = -0.010348
Iteration 21	 obj = -0.010349
Iteration 22	 obj = -0.010350
Iteration 23	 obj = -0.010350
Iteration 24	 obj = -0.010326
Iteration 25	 obj = -0.010357
Iteration 26	 obj = -0.010361
Iteration 27	 obj = -0.010365
Iteration 28	 obj = -0.010368
Iteration 29	 obj = -0.010370
Iteration 30	 obj = -0.010359
Iteration 31	 obj = -0.010377
Iteration 32	 obj = -0.010386
Iteration 33	 obj =